# Lab 1.3.2: Matrix Multiplication - SOLUTIONS

This notebook contains complete solutions to the exercises in Lab 1.3.2.

In [ ]:
import numpy as np
import time
from numba import cuda, float32

print(f"CUDA available: {cuda.is_available()}")

## Solution: 32×32 Tiled Matrix Multiplication

In [ ]:
TILE_SIZE_32 = 32

@cuda.jit
def matmul_tiled_32_kernel(A, B, C):
    """
    SOLUTION: Tiled matmul with 32×32 tiles.
    
    Benefits:
    - More data reuse (32×32 = 1024 elements per tile vs 256)
    - Full utilization of 1024 threads per block
    - Better memory bandwidth efficiency
    """
    # Shared memory for larger tiles
    sA = cuda.shared.array(shape=(32, 32), dtype=float32)
    sB = cuda.shared.array(shape=(32, 32), dtype=float32)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y
    
    row = by * 32 + ty
    col = bx * 32 + tx
    
    M, K = A.shape
    K2, N = B.shape
    
    tmp = float32(0.0)
    num_tiles = (K + 31) // 32
    
    for tile_idx in range(num_tiles):
        # Load tile of A
        a_col = tile_idx * 32 + tx
        if row < M and a_col < K:
            sA[ty, tx] = A[row, a_col]
        else:
            sA[ty, tx] = 0.0
        
        # Load tile of B
        b_row = tile_idx * 32 + ty
        if b_row < K and col < N:
            sB[ty, tx] = B[b_row, col]
        else:
            sB[ty, tx] = 0.0
        
        cuda.syncthreads()
        
        # Compute partial dot product
        for k in range(32):
            tmp += sA[ty, k] * sB[k, tx]
        
        cuda.syncthreads()
    
    if row < M and col < N:
        C[row, col] = tmp


def matmul_gpu_32(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    """Matrix multiplication using 32×32 tiles."""
    M, K = A.shape
    K2, N = B.shape
    assert K == K2
    
    d_A = cuda.to_device(A)
    d_B = cuda.to_device(B)
    d_C = cuda.device_array((M, N), dtype=np.float32)
    
    threads_per_block = (32, 32)  # 1024 threads per block (max!)
    blocks_x = (N + 31) // 32
    blocks_y = (M + 31) // 32
    blocks = (blocks_x, blocks_y)
    
    matmul_tiled_32_kernel[blocks, threads_per_block](d_A, d_B, d_C)
    
    return d_C.copy_to_host()

In [ ]:
# Test the solution
np.random.seed(42)
A = np.random.randn(1024, 1024).astype(np.float32)
B = np.random.randn(1024, 1024).astype(np.float32)

C_reference = A @ B
C_gpu = matmul_gpu_32(A, B)

print("32×32 Tiled Matrix Multiplication Test")
print("="*50)
print(f"Matrix size: 1024×1024")
print(f"Correct: {np.allclose(C_gpu, C_reference, rtol=1e-4)}")

In [ ]:
# Benchmark comparison: 16×16 vs 32×32
TILE_SIZE_16 = 16

@cuda.jit
def matmul_tiled_16_kernel(A, B, C):
    """16×16 tiled version for comparison."""
    sA = cuda.shared.array(shape=(16, 16), dtype=float32)
    sB = cuda.shared.array(shape=(16, 16), dtype=float32)
    
    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bx = cuda.blockIdx.x
    by = cuda.blockIdx.y
    
    row = by * 16 + ty
    col = bx * 16 + tx
    
    M, K = A.shape
    K2, N = B.shape
    
    tmp = float32(0.0)
    num_tiles = (K + 15) // 16
    
    for tile_idx in range(num_tiles):
        a_col = tile_idx * 16 + tx
        if row < M and a_col < K:
            sA[ty, tx] = A[row, a_col]
        else:
            sA[ty, tx] = 0.0
        
        b_row = tile_idx * 16 + ty
        if b_row < K and col < N:
            sB[ty, tx] = B[b_row, col]
        else:
            sB[ty, tx] = 0.0
        
        cuda.syncthreads()
        
        for k in range(16):
            tmp += sA[ty, k] * sB[k, tx]
        
        cuda.syncthreads()
    
    if row < M and col < N:
        C[row, col] = tmp


# Benchmark
print("\nBenchmark: 16×16 vs 32×32 Tiles")
print("="*50)

d_A = cuda.to_device(A)
d_B = cuda.to_device(B)
d_C = cuda.device_array((1024, 1024), dtype=np.float32)

# 16×16 tiles
for _ in range(3):
    matmul_tiled_16_kernel[(64, 64), (16, 16)](d_A, d_B, d_C)
cuda.synchronize()

start = time.perf_counter()
for _ in range(10):
    matmul_tiled_16_kernel[(64, 64), (16, 16)](d_A, d_B, d_C)
cuda.synchronize()
time_16 = (time.perf_counter() - start) / 10

# 32×32 tiles
for _ in range(3):
    matmul_tiled_32_kernel[(32, 32), (32, 32)](d_A, d_B, d_C)
cuda.synchronize()

start = time.perf_counter()
for _ in range(10):
    matmul_tiled_32_kernel[(32, 32), (32, 32)](d_A, d_B, d_C)
cuda.synchronize()
time_32 = (time.perf_counter() - start) / 10

flops = 2 * 1024 * 1024 * 1024
print(f"16×16 tiles: {time_16*1000:.3f} ms, {flops/time_16/1e9:.1f} GFLOPS")
print(f"32×32 tiles: {time_32*1000:.3f} ms, {flops/time_32/1e9:.1f} GFLOPS")
print(f"\n32×32 is {time_16/time_32:.2f}x faster than 16×16")

## Bonus: Register Blocking Concept

Each thread computes a small tile (e.g., 2×2 or 4×4) of output:

In [ ]:
# Conceptual code - register blocking
print("""
Register Blocking Concept:

Instead of each thread computing 1 output element:
  Thread 0: C[0,0]
  Thread 1: C[0,1]
  ...

Each thread computes a 2×2 block:
  Thread 0: C[0,0], C[0,1], C[1,0], C[1,1]
  Thread 1: C[0,2], C[0,3], C[1,2], C[1,3]
  ...

Benefits:
- 4 outputs per thread = 4× more arithmetic per memory load
- Better arithmetic intensity (compute/memory ratio)
- Uses registers more efficiently
- This is how cuBLAS achieves peak performance!

Trade-off:
- More registers per thread = lower occupancy
- But often worth it for compute-bound kernels
""")

## Cleanup

In [ ]:
import gc
del A, B, C_reference, C_gpu, d_A, d_B, d_C
gc.collect()
cuda.current_context().reset()
print("✅ Cleanup complete")